# Python Excel Handling Basics
Because you will encounter excel - no matter if you want or not

This notebook covers:

- text alignment (correctly display newlines in xlsx files exported from pandas)
- set cell borders & fonts
- set cell widths

In [1]:
"""
pip install openpyxl==3.1.5
pip install pandas==2.2.3
pip install xlsxwriter==3.2.0
"""

'\npip install openpyxl==3.1.5\npip install pandas==2.2.3\npip install xlsxwriter==3.2.0\n'

In [2]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment


# general parameters
save_file = "aligned.xlsx"

# create DataFrame
df = pd.DataFrame({"texts": ["line1\nline2", "line1"], "numbers": [1, 2], "long header name": [3, 4]})



# Write DataFrame with defined header and body formatting and cell widths
with pd.ExcelWriter(save_file, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="MySheet", index=False)
    workbook = writer.book
    worksheet = writer.sheets["MySheet"]
    
    # define & apply header formatting (bold, bottom horizontal line, vertical lines)
    header_format = workbook.add_format({
        "bold": True,
        "bottom": 2,  # bold cell bottom lines
        "left": 1  # thin left cell lines
    })
    for col_num, col_header in enumerate(df.columns.values):
        worksheet.set_column(col_num, col_num, len(col_header) + 1)  # cell width aligned to header length 
        worksheet.write(0, col_num, col_header, header_format)  # header format

    # define & apply body formatting (vertical lines)
    cell_format = workbook.add_format({
        "left": 1 
    })
    for row in range(1, len(df) + 1):
        for col in range(len(df.columns)):
            worksheet.write(row, col, df.iloc[row - 1, col], cell_format)



# fix string cell alignment (newlines)
to_align_cols = ["texts"]
col_ids = [id + 1 for id in range(len(df.columns)) if df.columns[id] in to_align_cols]
wb = load_workbook(save_file)
ws = wb.active
for col_id in col_ids:
    for row_id in range(len(df)):
        cell = ws.cell(row=row_id+2, column=col_id)
        cell.alignment = Alignment(wrap_text=True)
wb.save(save_file)